In [1]:
# _September_Week1 2018: 9/11/18-9/26/18

import pandas as pd
import numpy as np
import datetime
import pandas as pd
from googleads import adwords
import google_analytics_reporting_api as ga
import os

week="_September_Week1" 

Start_Date_overall='2018-09-11'
End_Data_overall='2018-09-26'

Start_Date_1='2018-09-11'
End_Date_1='2018-09-18'

Start_Date_2='2018-09-19'
End_Date_2='2018-09-26'


today_str=str(datetime.datetime.now().date())
writer_folder="/home/jian/Projects/Saatva/To_Lexie_and_Connor/output/running_on_"+today_str+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

writer_folder="/home/jian/Projects/Saatva/To_Lexie_and_Connor/output/running_on_"+today_str+"/"+week[1:]+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

# Google Analytics

In [2]:
analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Saatva/GA/GA_api_jubapluscc@gmail.com.json')

def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].apply(lambda x: int(x.replace(",","")))
        df['ga:sessions']=df['ga:sessions'].astype(int)

    if 'ga:adClicks' in df.columns:
        df['ga:adClicks']=df['ga:adClicks'].apply(lambda x: int(x.replace(",","")))
        df['ga:adClicks']=df['ga:adClicks'].astype(int)

    if 'ga:impressions' in df.columns:
        df['ga:impressions']=df['ga:impressions'].apply(lambda x: int(x.replace(",","")))
        df['ga:impressions']=df['ga:impressions'].astype(int)
    
    if 'ga:transactions' in df.columns:
        df['ga:transactions']=df['ga:transactions'].apply(lambda x: int(x.replace(",","")))
        df['ga:transactions']=df['ga:transactions'].astype(int)
    
    if 'ga:transactionRevenue' in df.columns:
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].apply(lambda x: float(x.replace(",","")))
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].astype(float)

    
    return df


In [18]:
# Saatva.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:sessions'],
                                            ga_dimensions=['ga:adwordsCustomerID'])

ga_data_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:sessions'],
                                            ga_dimensions=['ga:adwordsCustomerID'])


ga_data_saatva=ga_data_1.append(ga_data_2)
ga_data_saatva['View']="Saatva.com"
ga_data_saatva['View_ID']="122377905"
ga_data_saatva.shape

(7, 4)

In [19]:
ga_data_saatva.head(20)

,ga:adwordsCustomerID,ga:sessions,View,View_ID
0,(not set),5955,Saatva.com,122377905
1,5088479292,6278,Saatva.com,122377905
2,5466299909,6379,Saatva.com,122377905
...,...,...,...,...
1,5088479292,26158,Saatva.com,122377905
2,5466299909,6659,Saatva.com,122377905
3,6340610672,1,Saatva.com,122377905


In [21]:
len(ga_data_saatva['ga:adwordsCustomerID'].unique())

4

In [9]:
# Saatvamattress.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:sessions'],
                                            ga_dimensions=['ga:dateHourMinute','ga:adwordsCustomerID'])

ga_data_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:sessions'],
                                            ga_dimensions=['ga:dateHourMinute','ga:adwordsCustomerID'])


ga_data_SM=ga_data_1.append(ga_data_2)
ga_data_SM['View']="Saatvamattress.com"
ga_data_SM['View_ID']="33605697"
ga_data_SM.shape

(64975, 5)

In [10]:
len(ga_data_SM['ga:adwordsCustomerID'].unique())

5

In [11]:
ga_data_SM['ga:adwordsCustomerID'].unique()

array(['(not set)', '6340610672', '7992086489', '6313656046', '1838453210'], dtype=object)

In [38]:
ga_data=ga_data_SM.append(ga_data_saatva)
ga_data=num_func_trans(ga_data)

In [39]:
ga_data['Source']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[0])
ga_data['Medium']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[1])

In [40]:
ga_data['Medium'].unique()

array(['(none)', 'partner', 'cpc', 'organic', 'display', 'referral',
       'social', 'CPC', 'amyann', 'microsite', 'email', 'podcast',
       'influencer', 'print', 'mobileposse-metroweb', 'mattressguides',
       'sleepopolis', 'ebates2', '(not set)', 'company_profile',
       'brandxads', 'robatkinson0687', 'kiwireport-desktop', 'paid social',
       'nydailynews-nydailynews', 'influencers', 'Product', 'partners'], dtype=object)

In [41]:
# msn in 'social'
# mail.yahoo.com / referral in 'referral'

ga_data['Defined_Media']=np.nan
ga_data['Defined_Media']=np.where(ga_data['Medium']=="partner","partner",
                                  np.where((ga_data['ga:sourceMedium'].str.contains('youtube.') | (ga_data['Medium']=="social")),"Social",
                                           np.where((ga_data['Source'].isin(['msn','instagram','pinterest','facebook'])) & (ga_data['Medium']=='cpc'),"Social", # Social: 'msn','instagram','pinterest','facebook
                                                    np.where(ga_data['ga:sourceMedium'].str.contains('criteo'),"Retarget",
                                                             np.where(ga_data['Source']=="(direct)","Direct",
                                                                      np.where((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='cpc'),"SEM",
                                                                               np.where((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='organic'),"SEO",
                                                                                        np.where(ga_data['Source']=="powerInbox","Email",
                                                                                                 np.where((ga_data['ga:sourceMedium'].str.contains('email')) & (~ga_data['ga:sourceMedium'].isin(["taboola / townnews-wvgazettemail","query.hemailaccessonline.com / referral"]) | (ga_data['ga:sourceMedium'].str.contains("mail.") )),"Email",
                                                                                                          np.where(ga_data['ga:sourceMedium'].str.contains('taboola'),"Video",
                                                                                                                   np.where(ga_data['Medium']=="referral","Referral",
                                                                                                                            np.where(ga_data['ga:sourceMedium'].str.contains('display'),"Display","Others")
                                                                                                                            )
                                                                                                                   )
                                                                                                          )
                                                                                                )
                                                                                       )
                                                                              )
                                                                     )
                                                            )
                                                   )
                                          )
                                 )


In [42]:
GA_taxonomy=ga_data[['ga:sourceMedium','Source','Medium','Defined_Media']].drop_duplicates()
GA_taxonomy.reset_index(inplace=True)
GA_taxonomy=GA_taxonomy.sort_values('Defined_Media')
del GA_taxonomy['index']

In [43]:
ga_data.head(2)

,ga:adClicks,ga:dateHour,ga:impressions,ga:sessions,ga:sourceMedium,ga:transactionRevenue,ga:transactions,View,View_ID,Source,Medium,Defined_Media
0,0,2018091100,0,58,(direct) / (none),0.0,0,Saatvamattress.com,33605697,(direct),(none),Direct
1,0,2018091100,0,28,(not set) / partner,1108.0,1,Saatvamattress.com,33605697,(not set),partner,partner


In [44]:
ga_agg_session=ga_data.groupby(['ga:dateHour','Defined_Media'])['ga:sessions'].sum().to_frame()
ga_agg_session.reset_index(inplace=True)
ga_agg_wide_session=ga_agg_session.pivot('ga:dateHour','Defined_Media','ga:sessions')
ga_agg_wide_session.reset_index(inplace=True)
ga_agg_wide_session['time_hour']=ga_agg_wide_session['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))

In [45]:
ga_agg_transaction=ga_data.groupby(['ga:dateHour','Defined_Media'])['ga:transactions'].sum().to_frame()
ga_agg_transaction.reset_index(inplace=True)
ga_agg_wide_trans=ga_agg_transaction.pivot('ga:dateHour','Defined_Media','ga:transactions')
ga_agg_wide_trans.reset_index(inplace=True)
ga_agg_wide_trans['time_hour']=ga_agg_wide_trans['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))

In [46]:
ga_agg_Revenue=ga_data.groupby(['ga:dateHour','Defined_Media'])['ga:transactionRevenue'].sum().to_frame()
ga_agg_Revenue.reset_index(inplace=True)
ga_agg_wide_Revenue=ga_agg_Revenue.pivot('ga:dateHour','Defined_Media','ga:transactionRevenue')
ga_agg_wide_Revenue.reset_index(inplace=True)
ga_agg_wide_Revenue['time_hour']=ga_agg_wide_Revenue['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))


In [47]:
writer=pd.ExcelWriter(writer_folder+'/Saatva_S_and_SM_GA_by_Media_'+Start_Date_1+'_to_'+End_Date_2+week+'.xlsx', engine='xlsxwriter')

In [48]:
# Combined both Saatva.com and Saatvamattress.com
ga_agg_wide_session.to_excel(writer,"Session_hour_media",index=False)
ga_agg_wide_trans.to_excel(writer,"Trans_hour_media",index=False)
ga_agg_wide_Revenue.to_excel(writer,"Revenue_hour_media",index=False)

ga_data.to_excel(writer,"data",index=False)
GA_taxonomy.to_excel(writer,"taxonomy",index=False)
writer.save()

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51320554757&gca_matchtype=p&gca_network=s&gca_device=c&gca_adposition=1t4&gca_loc_interest_ms=&gca_loc_physical_ms=9061285&gclid=EAIaIQobChMI6ObC9u-y3QIVQrXACh379gBNEAAYBCAAEgJgMfD_BwE%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51627636938&gca_matchtype=e&gca_network=s&gca_device=c&gca_adposition=1s1&gca_loc_interest_ms=&gca_loc_physical_ms=9016899&gclid=EAIaIQobChMI0P7lhfGy3QIVELnACh3rTAr1EAEYASAAEgKzQPD_BwE%20/%20partner' with link or location/anchor > 255 cha

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://googleads.g.doubleclick.net/pagead/ads?client=ca-pub-5623576687264665&output=html&h=250&slotname=7500739833&adk=3746378559&adf=3683299514&w=346&lmt=1537104090&loeid=40993911&rafmt=12&guci=2.2.0.0.2.2.0&format=346x250&url=https://www.buzzfond.com/this-man-found-a-baby-dying-in-the-sun-but-he-had-no-idea-what-it-would-grow-into-what-he-did-next-defied-all-odds/2/&flash=0&wgl=1&dt=1537104090057&bpp=34&bdt=160&fdt=41&idt=51&shv=r20180910&cbv=r20180604&saldr=aa&abxe=1&correlator=602260693236&frm=20&pv=2&ga_vid=300156984.1537103905&ga_sid=1537104090&ga_hid=2016309516&ga_fc=0&iag=0&icsg=8874&dssz=13&mdo=0&mso=0&u_tz=-240&u_his=2&u_java=0&u_h=640&u_w=360&u_ah=640&u_aw=360&u_cd=32&u_nplug=0&u_nmime=0&adx=7&ady=109&biw=360&bih=560&scr_x=0&scr_y=0&eid=21060853,40993901,368226500&oid=3&ref=https://www.buzzfond.com/this-man-found-a-baby-dying-in-the-sun-but-he-had-no-idea-what-it-would-grow-into-w

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=Cj0KCQjwrZLdBRCmARIsAFBZllGMW6Ldgl_9_wiH8Ne7UA06dnGOPN--RCVeZFNw376F72bzBJrLsbIaApVQEALw_wcB%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=Cj0KCQjwrZLdBRCmARIsAFBZllFoTGL_4Xxmq59xbYC910oGhoGr1QnP-Om5fA-_F8rHhl8mmeCCF6UaAle0EALw_wcB%20/%20partner' with link or locat

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress%20reviews&gca_campaignid=1016369000&gca_adgroupid=51627636938&gca_matchtype=e&gca_network=s&gca_device=c&gca_adposition=1t1&gca_loc_interest_ms=&gca_loc_physical_ms=9012149&gclid=EAIaIQobChMIlqPensnY3QIVCZBpCh0NWQB1EAAYASAAEgKqa_D_BwE%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=Cj0KCQjw3KzdBRDWARIsAIJ8TMSguW4s-EFNYGSH-cKYTbgybIaDKIfeEHcLrKPU9-1jN8PkT9GjodsaAuuoEALw_wcB%20/%20partner' with link or locat

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51320554757&gca_matchtype=p&gca_network=s&gca_device=c&gca_adposition=1t4&gca_loc_interest_ms=&gca_loc_physical_ms=9061285&gclid=EAIaIQobChMI6ObC9u-y3QIVQrXACh379gBNEAAYBCAAEgJgMfD_BwE' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51627636938&gca_matchtype=e&gca_network=s&gca_device=c&gca_adposition=1s1&gca_loc_interest_ms=&gca_loc_physical_ms=9016899&gclid=EAIaIQobChMI0P7lhfGy3QIVELnACh3rTAr1EAEYASAAEgKzQPD_BwE' with link or location/anchor > 255 characters since it exceeds Exc

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://googleads.g.doubleclick.net/pagead/ads?client=ca-pub-2003873426348785&output=html&h=266&slotname=7019590105&adk=3167757419&adf=3222090758&w=320&fwr_io=true&lmt=1537110187&rafmt=1&guci=2.2.0.0.2.2.0&format=320x266&url=https://freeprizesonline.com/2018/09/16/baby-teether-food-grade-baby-silicone-turtle-animals-yellow/&flash=0&fwr=1&fwrattr=true&rh=0&rw=272&resp_fmts=3&sfro=1&wgl=1&dt=1537110187717&bpp=7&bdt=292&fdt=166&idt=168&shv=r20180910&cbv=r20180604&saldr=aa&abxe=1&prev_fmts=0x0,320x266&nras=1&correlator=2067091884799&frm=20&pv=1&ga_vid=179881647.1537110188&ga_sid=1537110188&ga_hid=188291779&ga_fc=0&iag=0&icsg=11811138239&dssz=31&mdo=0&mso=0&u_tz=-240&u_his=1&u_java=0&u_h=658&u_w=320&u_ah=658&u_aw=320&u_cd=24&u_nplug=0&u_nmime=0&adx=0&ady=952&biw=320&bih=538&scr_x=0&scr_y=0&eid=21060853,828064256,828064254&oid=3&ref=http://m.facebook.com/&rx=0&eae=0&fc=1940&brdim=0,0,0,0,320,0,320,

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51320554757&gca_matchtype=p&gca_network=g&gca_device=c&gca_adposition=1t4&gca_loc_interest_ms=&gca_loc_physical_ms=9003922&gclid=CjwKCAjwxILdBRBqEiwAHL2R8zxtFU5lW7SYjzprq7voFgWnHDqgeWQFApLyR9zqrP7m5rQp9wV0YxoCmO0QAvD_BwE' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://googleads.g.doubleclick.net/pagead/ads?client=ca-pub-4707593535629137&output=html&h=280&slotname=9267576712&adk=1858352339&adf=2425008757&w=336&lmt=1537298275&pucrd=CgwIABAAGAMgACgAOAESAhgHeAM&guci=2.2.0.0.2.2.0&format=336x280&url=https://juliesfreebies.com/sweepstake/the-mcdonalds-arch-card-instant-win-game/&flash=0&wgl=1&adsid=AHva7ovEx

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=CjwKCAjw85zdBRB6EiwAov3RiqgAL1hcM2KcraQURhJx6XlsedwZ6s2Xc_aPMOaY7a6i2LVL4jR4JRoCSFcQAvD_BwE' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=EAIaIQobChMIxNaJ8YjS3QIVkICfCh2PqwvKEAAYASAAEgJdtfD_BwE' with link or location/anchor > 255 characters since it exceeds Excel's limit for URL

In [49]:
GA_taxonomy.head(2)

,ga:sourceMedium,Source,Medium,Defined_Media
0,(direct) / (none),(direct),(none),Direct
375,atlastargeting / display,atlastargeting,display,Display


In [50]:
GA_taxonomy.to_csv(writer_folder+'/Saatva_S_and_SM_GA_by_Media_Taxonomy_'+Start_Date_1+'_to_'+End_Date_2+week+'.csv')

In [51]:
media_col=ga_agg_wide_session.columns.tolist()[1:(len(ga_agg_wide_session.columns)-1)]

In [52]:
def write_csv_output(df,Matrics):
    ga_output_csv=df.copy()
    ga_output_csv['Date']=ga_output_csv['ga:dateHour'].apply(lambda x:int(x[0:8]))
    ga_output_csv['Hour']=ga_output_csv['ga:dateHour'].apply(lambda x:int(x[8:10]))
    del ga_output_csv['ga:dateHour']
    ga_output_csv=ga_output_csv.sort_values(['Date','Hour'],ascending=True)
    ga_output_csv=ga_output_csv[['Date','Hour']+media_col]
    ga_output_csv=ga_output_csv.fillna(0)
    ga_output_csv.to_csv(writer_folder+'/Saatva_S_and_SM_GA_by_Media_'+Start_Date_1+'_to_'+End_Date_2+week+"_"+Matrics+'.csv',index=False)

In [53]:
write_csv_output(ga_agg_wide_session,"Session")
write_csv_output(ga_agg_wide_trans,"Trans")
write_csv_output(ga_agg_wide_Revenue,"Revenue")


In [54]:
ga_agg_session_by_View=ga_data.groupby(['View','View_ID','ga:dateHour','Defined_Media'])['ga:sessions'].sum().unstack('Defined_Media')
ga_agg_session_by_View.reset_index(inplace=True)
ga_agg_session_by_View=ga_agg_session_by_View.fillna(0)
ga_agg_session_by_View.to_csv(writer_folder+'/Saatva_S_and_SM_GA_by_Media_by_View'+Start_Date_1+'_to_'+End_Date_2+week+"_Sessions"+'.csv',index=False)




# AdWords keywords from GA

In [55]:
from fuzzywuzzy import fuzz
# fuzz 60 ratio score thredhold
writer=pd.ExcelWriter(writer_folder+"KeyWords_From_GA+"+Start_Date_1+'_to_'+End_Date_2+week+".xlsx")

In [56]:
# Saatva.com
# ga:date ga:keyword ga:impressions ga:adClicks
ga_data_Ads_Saatva_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:adClicks','ga:impressions'],
                                            ga_dimensions=['ga:date','ga:keyword'])
ga_data_Ads_Saatva_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:adClicks','ga:impressions'],
                                            ga_dimensions=['ga:date','ga:keyword'])
ga_data_Ads_Saatva=ga_data_Ads_Saatva_1.append(ga_data_Ads_Saatva_2)
ga_data_Ads_Saatva['key_word_list']=ga_data_Ads_Saatva['ga:keyword'].apply(lambda x: x.split(" "))
ga_data_Ads_Saatva['Brand_Ratio']=ga_data_Ads_Saatva['key_word_list'].apply(lambda x: max([fuzz.ratio("saatva",z) for z in x]))
ga_data_Ads_Saatva['key_words_group']=np.where(ga_data_Ads_Saatva['Brand_Ratio']>60,"Brand","Non_Brand")

taxonomy_Saatva=ga_data_Ads_Saatva[['ga:keyword','key_words_group']].drop_duplicates()
taxonomy_Saatva['website']="Saatva.com"
ga_data_Ads_Saatva['ga:adClicks']=ga_data_Ads_Saatva['ga:adClicks'].astype(int)
ga_data_Ads_Saatva['ga:impressions']=ga_data_Ads_Saatva['ga:impressions'].astype(int)
output_Saatva=ga_data_Ads_Saatva.groupby(["ga:date","ga:keyword","key_words_group"])['ga:adClicks','ga:impressions'].sum().reset_index()
output_Saatva_Brand=output_Saatva[output_Saatva['key_words_group']=="Brand"].sort_values('ga:date').reset_index()
del output_Saatva_Brand['index']

output_Saatva_Brand.to_excel(writer,"Saatva_Brand",index=False)
output_Saatva.to_excel(writer,"Saatva_Keywords",index=False)

In [57]:
# SaatvaMattress.com
# ga:date ga:keyword ga:impressions ga:adClicks
ga_data_Ads_SaatvaMattress_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:adClicks','ga:impressions'],
                                            ga_dimensions=['ga:date','ga:keyword'])
ga_data_Ads_SaatvaMattress_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:adClicks','ga:impressions'],
                                            ga_dimensions=['ga:date','ga:keyword'])
ga_data_Ads_SaatvaMattress=ga_data_Ads_SaatvaMattress_1.append(ga_data_Ads_SaatvaMattress_2)
ga_data_Ads_SaatvaMattress['key_word_list']=ga_data_Ads_SaatvaMattress['ga:keyword'].apply(lambda x: x.split(" "))
ga_data_Ads_SaatvaMattress['Brand_Ratio']=ga_data_Ads_SaatvaMattress['key_word_list'].apply(lambda x: max([fuzz.ratio("saatva",z) for z in x]))
ga_data_Ads_SaatvaMattress['key_words_group']=np.where(ga_data_Ads_SaatvaMattress['Brand_Ratio']>60,"Brand","Non_Brand")

taxonomy_SaatvaMattress=ga_data_Ads_SaatvaMattress[['ga:keyword','key_words_group']].drop_duplicates()
taxonomy_SaatvaMattress['website']="SaatvaMattress.com"
ga_data_Ads_SaatvaMattress['ga:adClicks']=ga_data_Ads_SaatvaMattress['ga:adClicks'].astype(int)
ga_data_Ads_SaatvaMattress['ga:impressions']=ga_data_Ads_SaatvaMattress['ga:impressions'].astype(int)
output_SaatvaMattress=ga_data_Ads_SaatvaMattress.groupby(["ga:date","ga:keyword","key_words_group"])['ga:adClicks','ga:impressions'].sum().reset_index()
output_SaatvaMattress_Brand=output_SaatvaMattress[output_SaatvaMattress['key_words_group']=="Brand"].sort_values('ga:date').reset_index()
del output_SaatvaMattress_Brand['index']

output_SaatvaMattress_Brand.to_excel(writer,"SaatvaMattress_Brand",index=False)
output_SaatvaMattress.to_excel(writer,"SaatvaMattress_Keywords",index=False)

In [58]:
taxonomy=taxonomy_Saatva.append(taxonomy_SaatvaMattress)
taxonomy.to_excel(writer,"taxonomy",index=False)
writer.save()

# Google AdWords

In [59]:
def get_client(config_path='googleads_saatva.yaml'):
    return adwords.AdWordsClient.LoadFromStorage(config_path)

def download_report_as_stream(client, report_type, fields, start_date, end_date):
    '''Download Adwords report, then save as csv file to base_dir
    Args:
        client: adwords.AdWordsClient.LoadFromStorage()
        report_type: str eg. 'KEYWORDS_PERFORMANCE_REPORT'
        fields: list eg. ["CampaignId", "CampaignName", ... ]
        start_date: str, eg. '2018-02-11'
        end_date: str, eg.
    Return:
        str, path to csv file
    '''
    report_downloader = client.GetReportDownloader(version='v201802')
    # Create report definition.
    report = {
        'reportName': report_type,
        'dateRangeType': 'CUSTOM_DATE',
        'reportType': report_type,
        'downloadFormat': 'CSV',
        'selector': {
            'fields': fields,
            'dateRange': {
                'min': start_date,
                'max': end_date}
        }
    }

    # save output
    return report_downloader.DownloadReportAsStream(report, 
        skip_report_header=True, skip_column_header=False,
        skip_report_summary=True)



In [60]:
client_Saatva = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva = download_report_as_stream(client_Saatva, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Click = pd.read_csv(response_Saatva)
Saatva_Click['Cost']=Saatva_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Click['Channel_Network']=Saatva_Click['Advertising Channel']+"|"+Saatva_Click['Network']+"|"+Saatva_Click['Network (with search partners)']

In [61]:
client_Saatva_Branding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Branding.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva_Branding = download_report_as_stream(client_Saatva_Branding, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Brand_Click = pd.read_csv(response_Saatva_Branding)
Saatva_Brand_Click['Cost']=Saatva_Brand_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Brand_Click['Channel_Network']=Saatva_Brand_Click['Advertising Channel']+"|"+Saatva_Brand_Click['Network']+"|"+Saatva_Brand_Click['Network (with search partners)']


In [62]:
client_Saatva_Competitors = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Competitors.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva_Competitor = download_report_as_stream(client_Saatva_Competitors, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Competitor_Click = pd.read_csv(response_Saatva_Competitor)
Saatva_Competitor_Click['Cost']=Saatva_Competitor_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Competitor_Click['Channel_Network']=Saatva_Competitor_Click['Advertising Channel']+"|"+Saatva_Competitor_Click['Network']+"|"+Saatva_Competitor_Click['Network (with search partners)']



In [63]:
client_TheSaatvaCompany = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_TheSaatvaCompany.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_TheSaatvaCompany = download_report_as_stream(client_TheSaatvaCompany, report_type, fields, Start_Date_overall, End_Data_overall)
TheSaatvaCompany_Click = pd.read_csv(response_TheSaatvaCompany)
TheSaatvaCompany_Click['Cost']=TheSaatvaCompany_Click['Cost'].apply(lambda x: x/1000000)
TheSaatvaCompany_Click['Channel_Network']=TheSaatvaCompany_Click['Advertising Channel']+"|"+TheSaatvaCompany_Click['Network']+"|"+TheSaatvaCompany_Click['Network (with search partners)']



In [64]:
client_TheSaatvaCompanyBranding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_TheSaatvaCompanyBranding.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_TheSaatvaCompanyBranding = download_report_as_stream(client_TheSaatvaCompanyBranding, report_type, fields, Start_Date_overall, End_Data_overall)
TheSaatvaCompanyBranding_Click = pd.read_csv(response_TheSaatvaCompanyBranding)
TheSaatvaCompanyBranding_Click['Cost']=TheSaatvaCompanyBranding_Click['Cost'].apply(lambda x: x/1000000)
TheSaatvaCompanyBranding_Click['Channel_Network']=TheSaatvaCompanyBranding_Click['Advertising Channel']+"|"+TheSaatvaCompanyBranding_Click['Network']+"|"+TheSaatvaCompanyBranding_Click['Network (with search partners)']



In [65]:
Saatva_Competitor_Click=Saatva_Competitor_Click[Saatva_Competitor_Click['Campaign'].apply(lambda x: x[len(x)-4:len(x)+1])=="[SM]"]
Saatva_Competitor_Click.reset_index(inplace=True)
del Saatva_Competitor_Click['index']
# Keep only campaign labeled as "[SM]" in Saatva Competitor account
# Broad and Exact ADL Non-Brand Search Partners are not included

In [66]:
set(Saatva_Click['Channel_Network'].unique().tolist()+Saatva_Brand_Click['Channel_Network'].unique().tolist()+Saatva_Competitor_Click['Channel_Network'].unique().tolist()+\
    TheSaatvaCompany_Click['Channel_Network'].unique().tolist()+TheSaatvaCompanyBranding_Click['Channel_Network'].unique().tolist())

{'Display|Display Network|Display Network',
 'Search|Search Network|Google search',
 'Search|Search Network|Search partners',
 'Shopping|Search Network|Google search',
 'Shopping|Search Network|Search partners',
 'Video|Display Network|Display Network',
 'Video|YouTube Search|YouTube Search',
 'Video|YouTube Videos|YouTube Videos'}

In [67]:
df_all_adwords=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click)
df_all_adwords=df_all_adwords.append(TheSaatvaCompany_Click).append(TheSaatvaCompanyBranding_Click)

In [90]:
df_s=df_all_adwords.groupby(['Channel_Network'])[['Impressions','Clicks','Cost']].sum().reset_index()
df_s[df_s['Channel_Network'].apply(lambda x: "Video" in x)]

,Channel_Network,Impressions,Clicks,Cost
5,Video|Display Network|Display Network,6540,17,34.395750
6,Video|YouTube Search|YouTube Search,165465,11,736.850000
7,Video|YouTube Videos|YouTube Videos,7159862,28307,93451.416854


In [85]:
df_all_adwords.head(2)

,Account,Campaign,Day,Hour of day,Advertising Channel,Network,Network (with search partners),Impressions,Clicks,Cost,Channel_Network
0,Saatva,[ADL Shopping] Saatva - {Neither} - (Desktop),2018-09-25,22,Shopping,Search Network,Google search,3055,20,110.43,Shopping|Search Network|Google search
1,Saatva,[ADL] [Display] [US] SCM - Conservative [CPA],2018-09-20,7,Display,Display Network,Display Network,360,1,0.95,Display|Display Network|Display Network


In [68]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|Display Network|Display Network"]['Cost'].sum()

34.39575

In [69]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Search|YouTube Search"]['Cost'].sum()

736.84999999999991

In [70]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Videos|YouTube Videos"]['Cost'].sum()

93451.41685400001

In [74]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|Display Network|Display Network"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).min()

datetime.date(2018, 9, 11)

In [75]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|Display Network|Display Network"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).max()

datetime.date(2018, 9, 19)

In [76]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Search|YouTube Search"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).min()

datetime.date(2018, 9, 11)

In [78]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Search|YouTube Search"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).max()

datetime.date(2018, 9, 26)

In [79]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Videos|YouTube Videos"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).min()

datetime.date(2018, 9, 11)

In [80]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Videos|YouTube Videos"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).max()

datetime.date(2018, 9, 26)

In [83]:
df_x=df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Videos|YouTube Videos"]
df_x['Day']=df_x['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
df_x[df_x['Day']<datetime.date(2018,9,19)]['Cost'].sum()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


9427.5978469999991

In [84]:
df_x[df_x['Day']<datetime.date(2018,9,19)]

,Account,Campaign,Day,Hour of day,Advertising Channel,Network,Network (with search partners),Impressions,Clicks,Cost,Channel_Network
3,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-18,23,Video,YouTube Videos,YouTube Videos,582,0,3.744637,Video|YouTube Videos|YouTube Videos
30,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-13,7,Video,YouTube Videos,YouTube Videos,100,0,0.634816,Video|YouTube Videos|YouTube Videos
31,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-11,7,Video,YouTube Videos,YouTube Videos,105,0,0.655700,Video|YouTube Videos|YouTube Videos
...,...,...,...,...,...,...,...,...,...,...,...
31847,Saatva,[ADL] [Video] [In-Stream] [US] Remarketing - L...,2018-09-17,8,Video,YouTube Videos,YouTube Videos,256,0,6.550561,Video|YouTube Videos|YouTube Videos
31886,Saatva,[ADL] [Video] [In-Stream] [US] Remarketing - L...,2018-09-16,16,Video,YouTube Videos,YouTube Videos,531,4,11.637798,Video|YouTube Videos|YouTube Videos
31890,Saatva,[ADL] [Video] [In-Stream] [US] Remarketing - L...,2018-09-18,23,Video,YouTube Videos,YouTube Videos,546,1,9.852961,Video|YouTube Videos|YouTube Videos


In [91]:
'''
Saatva_Click=Saatva_Click[~((Saatva_Click['Campaign'].str.contains("Loom")) | (Saatva_Click['Campaign'].str.contains("Leaf")) | (Saatva_Click['Campaign'].str.contains("Zenhaven")))]
TheSaatvaCompany_Click=TheSaatvaCompany_Click[~((TheSaatvaCompany_Click['Campaign'].str.contains("Loom")) | (TheSaatvaCompany_Click['Campaign'].str.contains("Leaf")) | (TheSaatvaCompany_Click['Campaign'].str.contains("Zenhaven")))]
TheSaatvaCompanyBranding_Click=TheSaatvaCompanyBranding_Click[~((TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Loom")) | (TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Leaf")) | (TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Zenhaven")))]
'''
# No need to exclude the compaigns because of the small pctg

Saatva_Click['Campaign_Type']=Saatva_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
Saatva_Brand_Click['Campaign_Type']=Saatva_Brand_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM_Brand").replace("Shopping","PLA"))
Saatva_Competitor_Click['Campaign_Type']="SEM_Non_Brand" # Because it's all labeled as Non-Brand in Campaign and in Search Only Campaign Type
TheSaatvaCompany_Click['Campaign_Type']=TheSaatvaCompany_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
TheSaatvaCompanyBranding_Click['Campaign_Type']=TheSaatvaCompanyBranding_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))


In [92]:
# Split Saatva
# 'Search Network with Display Select' Campaign Type all 0 Impr and Clicks
Saatva_Click_Search=Saatva_Click[Saatva_Click['Advertising Channel']=='Search']
Saatva_Click_Others=Saatva_Click[Saatva_Click['Advertising Channel']!='Search']

Saatva_Click_Search['Campaign_Type']=np.where((Saatva_Click_Search['Campaign'].str.contains("Branded")) | (Saatva_Click_Search['Campaign'].str.contains(" Brand ")) | (Saatva_Click_Search['Campaign'].str.contains("Co-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((Saatva_Click_Search['Campaign'].str.contains("Non-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
Saatva_Click_Search_1=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']!="NA"]
Saatva_Click_Search_2=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']=="NA"]
len(Saatva_Click_Search_2) # Make sure it's 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0

In [93]:
# Split TheSaatvaCompany

TheSaatvaCompany_Click_Search=TheSaatvaCompany_Click[TheSaatvaCompany_Click['Advertising Channel']=='Search']
TheSaatvaCompany_Click_Others=TheSaatvaCompany_Click[TheSaatvaCompany_Click['Advertising Channel']!='Search']

TheSaatvaCompany_Click_Search['Campaign_Type']=np.where((TheSaatvaCompany_Click_Search['Campaign'].str.contains("Branded")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains(" Brand ")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("Co-Brand")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((TheSaatvaCompany_Click_Search['Campaign'].str.contains("Non-Brand")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
TheSaatvaCompany_Click_Search_1=TheSaatvaCompany_Click_Search[TheSaatvaCompany_Click_Search['Campaign_Type']!="NA"]
TheSaatvaCompany_Click_Search_2=TheSaatvaCompany_Click_Search[TheSaatvaCompany_Click_Search['Campaign_Type']=="NA"]
len(TheSaatvaCompany_Click_Search_2) # Make sure it's 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0

In [94]:
# All campaing is able to differentiate from the name about brand and non-brand, so no need to go to keywords level
'''
Saatva_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva_keywords.csv",dtype=str,skiprows=2)
Saatva_Brand_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva branding_keywords.csv",dtype=str)
Saatva_Competitor_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva competitors_keywords.csv",dtype=str,skiprows=2)

Saatva_keywords=Saatva_keywords[~((Saatva_keywords['Campaign'].str.contains("Loom")) | (Saatva_keywords['Campaign'].str.contains("Leaf")) | (Saatva_keywords['Campaign'].str.contains("Zenhaven")))]
Saatva_keywords['Campaign_Type']=np.where(Saatva_keywords['Search keyword'].str.contains("aatva"),"SEM_Brand","SEM_Non_Brand")
Saatva_keywords['Impressions']=Saatva_keywords['Impressions'].apply(lambda x: int(x.replace(",","")))
Saatva_keywords['Impressions']=Saatva_keywords['Impressions'].astype(int)
Saatva_keywords['Clicks']=Saatva_keywords['Clicks'].apply(lambda x: int(x.replace(",","")))
Saatva_keywords['Clicks']=Saatva_keywords['Clicks'].astype(int)
Saatva_keywords=Saatva_keywords.groupby(['Campaign','Campaign_Type'])['Impressions','Clicks'].sum()
Saatva_keywords.reset_index(inplace=True)
Saatva_keywords=Saatva_keywords.sort_values(['Campaign','Impressions'])
Saatva_keywords=Saatva_keywords.drop_duplicates('Campaign')
Saatva_Campaign_Type=Saatva_keywords[['Campaign','Campaign_Type']]

del Saatva_Click_Search_2['Campaign_Type']
Saatva_Click_Search_2=pd.merge(Saatva_Click_Search_2,Saatva_Campaign_Type,on="Campaign",how="left")

Saatva_Search_Click=Saatva_Click_Search_1.append(Saatva_Click_Search_2)
Saatva_Search_Click['Campaign_Type']=Saatva_Search_Click['Campaign_Type'].fillna("All_0_Values_and_No_Kws")
'''

'\nSaatva_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva_keywords.csv",dtype=str,skiprows=2)\nSaatva_Brand_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva branding_keywords.csv",dtype=str)\nSaatva_Competitor_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva competitors_keywords.csv",dtype=str,skiprows=2)\n\nSaatva_keywords=Saatva_keywords[~((Saatva_keywords[\'Campaign\'].str.contains("Loom")) | (Saatva_keywords[\'Campaign\'].str.contains("Leaf")) | (Saatva_keywords[\'Campaign\'].str.contains("Zenhaven")))]\nSaatva_keywords[\'Campaign_Type\']=np.where(Saatva_keywords[\'Search keyword\'].str.contains("aatva"),"SEM_Brand","SEM_Non_Brand")\nSaatva_keywords[\'Impressions\']=Saatva_keywords[\'Impressions\'].apply(lambda x: int(x.replace(",","")))\nSaatva_keywords[\'Impressions\']=Saatva_keywords[\'Impressions\'].astype(int)\nSaatva_keywords[\'Clicks\']=Saatva_keywords[\'Clicks\'].apply(

In [95]:
Saatva_Click_Search_2.head(2)

,Account,Campaign,Day,Hour of day,Advertising Channel,Network,Network (with search partners),Impressions,Clicks,Cost,Channel_Network,Campaign_Type


In [96]:
Saatva_Click=Saatva_Click_Search.append(Saatva_Click_Others)
TheSaatvaCompany_Click=TheSaatvaCompany_Click_Search.append(TheSaatvaCompany_Click_Others)


In [97]:
writer=pd.ExcelWriter(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+"_cost.xlsx",engine='xlsxwriter')

In [98]:
Saatva_Agg=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click).append(TheSaatvaCompany_Click).append(TheSaatvaCompanyBranding_Click)
Saatva_Agg['Day']=Saatva_Agg['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Saatva_Agg.reset_index(inplace=True)
del Saatva_Agg['index']
Saatva_Agg['Hour of day']=Saatva_Agg['Hour of day'].astype(int)
Saatva_Agg_Category=Saatva_Agg[['Campaign','Account','Advertising Channel','Campaign_Type']].drop_duplicates()
Saatva_Agg_Category.to_excel(writer,"Campaign Category",index=False)

In [99]:
Saatva_Agg=Saatva_Agg.groupby(["Campaign_Type","Day","Hour of day"])['Impressions','Clicks', 'Cost'].apply(sum)
Saatva_Agg.reset_index(inplace=True)
Saatva_Agg['CTR']=Saatva_Agg['Clicks']/Saatva_Agg['Impressions']
Saatva_Agg.to_excel(writer,"reshaped by hour day",index=False)

Saatva_Agg_No_NA=Saatva_Agg[(Saatva_Agg['Campaign_Type']!="NA") & (Saatva_Agg['Campaign_Type']!="Search Network with Display Select")]

Saatva_Agg_No_NA.to_excel(writer,"clean data by hour day",index=False)

In [100]:
writer.save()

In [101]:
AdWords_output=Saatva_Agg.copy()
AdWords_output['Date']=AdWords_output['Day'].apply(lambda x: str(x)[0:4]+str(x)[5:7]+str(x)[8:10])
AdWords_output=AdWords_output[['Campaign_Type','Date','Hour of day','Impressions','Clicks','Cost']]
AdWords_output=AdWords_output.rename(columns={"Hour of day":"Hour","Impressions":"IMP"})
AdWords_output=AdWords_output.sort_values(["Campaign_Type","Date","Hour"],ascending=[True,True,True])
AdWords_output.to_csv(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+"_cost.csv",index=False)